In [2]:
# ================================
# HYBRID AWS ARCHITECTURAL RECOMMENDER
# ML (TF-IDF) + DistilBERT + Top-k Ranking & Subcategory Choice
# ================================

import pandas as pd
import numpy as np
import re
import joblib
from typing import Tuple, List

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Torch / Transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup,
)
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ================================
# LOAD DATASET + SERVICE CATEGORY MAP
# ================================

# Load full CSV (with service_category etc.)
df_full = pd.read_csv("aws_architectural_recommender_dataset_11000.csv")
print("Original shape:", df_full.shape)
print("Columns:", df_full.columns.tolist())

# Build service -> category mapping using ALL services present in dataset
service_category_map = (
    df_full[["primary_service", "service_category"]]
    .dropna()
    .drop_duplicates()
    .set_index("primary_service")["service_category"]
    .to_dict()
)
print("\nService → Category map (sample):")
for s, c in list(service_category_map.items())[:10]:
    print(f"  {s}  -->  {c}")

# Now keep only columns needed for training
df = df_full[["requirement_text", "primary_service"]].copy()

# ================================
# BASIC CLEANING
# ================================

df = df.dropna(subset=["requirement_text", "primary_service"])
df["requirement_text"] = df["requirement_text"].astype(str).str.strip()
df["primary_service"] = df["primary_service"].astype(str).str.strip()

df = df[df["requirement_text"].astype(bool)]
df = df[df["primary_service"].astype(bool)]
df.reset_index(drop=True, inplace=True)

print("\nAfter cleaning:", df.shape)
df.head()

def preprocess(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["requirement_text"].apply(preprocess)
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)

print("After preprocessing:", df.shape)

# ================================
# SPLIT DATA
# ================================

X = df["clean_text"]
y = df["primary_service"]

label_encoder = LabelEncoder()
y_ids = label_encoder.fit_transform(y)
num_labels = len(label_encoder.classes_)
print("Number of classes:", num_labels)
print("Classes (sample):", list(label_encoder.classes_)[:10], "...")

# 70% train, 30% temp
X_train, X_temp, y_train_ids, y_temp_ids = train_test_split(
    X,
    y_ids,
    test_size=0.3,
    random_state=42,
    stratify=y_ids
)

# 15% val, 15% test
X_val, X_test, y_val_ids, y_test_ids = train_test_split(
    X_temp,
    y_temp_ids,
    test_size=0.5,
    random_state=42,
    stratify=y_temp_ids
)

print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

y_train_str = label_encoder.inverse_transform(y_train_ids)
y_val_str   = label_encoder.inverse_transform(y_val_ids)
y_test_str  = label_encoder.inverse_transform(y_test_ids)

# ================================
# TF-IDF + ML MODELS
# ================================

vectorizer = TfidfVectorizer(
    max_features=7000,
    ngram_range=(1, 2),
    sublinear_tf=True,
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec   = vectorizer.transform(X_val)
X_test_vec  = vectorizer.transform(X_test)

print("TF-IDF shapes:", X_train_vec.shape, X_val_vec.shape, X_test_vec.shape)

ml_models = {
    "LinearSVC": LinearSVC(),
    "LogisticRegression": LogisticRegression(max_iter=5000, n_jobs=-1),
    "MultinomialNB": MultinomialNB(),
    "RandomForest": RandomForestClassifier(n_estimators=400, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
}

ml_results = {}

for name, model in ml_models.items():
    print(f"\nTraining ML model: {name}")
    model.fit(X_train_vec, y_train_str)
    y_pred = model.predict(X_val_vec)

    acc = accuracy_score(y_val_str, y_pred)
    f1  = f1_score(y_val_str, y_pred, average="weighted")

    ml_results[name] = (acc, f1)
    print(f"{name} Val Accuracy: {acc:.4f}")
    print(f"{name} Val F1-score: {f1:.4f}")
    print("-" * 50)

ml_results_df = pd.DataFrame(ml_results, index=["Accuracy", "F1"]).T
ml_results_df.sort_values("F1", ascending=False, inplace=True)
print("\nML validation leaderboard:\n", ml_results_df)

best_ml_name = ml_results_df["F1"].idxmax()
best_ml_model = ml_models[best_ml_name]
print(f"\nBest ML model (val): {best_ml_name}")

# Retrain best model on train+val
X_train_val_vec = vectorizer.fit_transform(pd.concat([X_train, X_val]))
y_train_val_str = np.concatenate([y_train_str, y_val_str])

best_ml_model.fit(X_train_val_vec, y_train_val_str)

# Save artifacts
joblib.dump(best_ml_model, "best_aws_classifier.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("Saved: best_aws_classifier.pkl, tfidf_vectorizer.pkl, label_encoder.pkl")

# Test
X_test_vec_final = vectorizer.transform(X_test)
y_test_pred_ml = best_ml_model.predict(X_test_vec_final)

ml_test_acc = accuracy_score(y_test_str, y_test_pred_ml)
ml_test_f1  = f1_score(y_test_str, y_test_pred_ml, average="weighted")

print("\n=== Best ML Model Test Performance ===")
print("Accuracy:", ml_test_acc)
print("Weighted F1:", ml_test_f1)
print("\nClassification Report (ML):\n")
print(classification_report(y_test_str, y_test_pred_ml))

# ================================
# DISTILBERT DATASETS
# ================================

df_bert = df[["requirement_text", "primary_service"]].copy()
df_bert["label_id"] = label_encoder.transform(df_bert["primary_service"])

train_mask = df["clean_text"].isin(X_train)
val_mask   = df["clean_text"].isin(X_val)
test_mask  = df["clean_text"].isin(X_test)

bert_train = df_bert[train_mask].reset_index(drop=True)
bert_val   = df_bert[val_mask].reset_index(drop=True)
bert_test  = df_bert[test_mask].reset_index(drop=True)

print("\nBERT Train size:", bert_train.shape)
print("BERT Val size:", bert_val.shape)
print("BERT Test size:", bert_test.shape)

class AWSDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text  = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

train_dataset = AWSDataset(
    texts=bert_train["requirement_text"],
    labels=bert_train["label_id"],
    tokenizer=tokenizer,
    max_length=128
)
val_dataset = AWSDataset(
    texts=bert_val["requirement_text"],
    labels=bert_val["label_id"],
    tokenizer=tokenizer,
    max_length=128
)
test_dataset = AWSDataset(
    texts=bert_test["requirement_text"],
    labels=bert_test["label_id"],
    tokenizer=tokenizer,
    max_length=128
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ================================
# DISTILBERT TRAINING
# ================================

model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label={i: l for i, l in enumerate(label_encoder.classes_)},
    label2id={l: i for i, l in enumerate(label_encoder.classes_)},
)
model.to(device)

EPOCHS = 5
LR = 2e-5

optimizer = AdamW(model.parameters(), lr=LR)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

def train_one_epoch(epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * input_ids.size(0)

        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total   += labels.size(0)

    avg_loss = total_loss / total
    acc = correct / total

    print(f"Epoch {epoch+1} Train Loss: {avg_loss:.4f} | Train Acc: {acc:.4f}")

from sklearn.metrics import f1_score

def eval_one_epoch():
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids      = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels         = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item() * input_ids.size(0)

            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    avg_loss = total_loss / total
    acc = correct / total
    f1  = f1_score(all_labels, all_preds, average="weighted")

    print(f"Val Loss: {avg_loss:.4f} | Val Acc: {acc:.4f} | Val F1: {f1:.4f}")
    return avg_loss, acc, f1

best_val_f1 = 0.0

for epoch in range(EPOCHS):
    print(f"\n===== Epoch {epoch+1}/{EPOCHS} =====")
    train_one_epoch(epoch)
    val_loss, val_acc, val_f1 = eval_one_epoch()

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        print("New best DistilBERT model, saving checkpoint...")
        model.save_pretrained("distilbert_aws_model")
        tokenizer.save_pretrained("distilbert_aws_model")

# ================================
# TEST DISTILBERT
# ================================

best_bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert_aws_model")
best_bert_model.to(device)
best_bert_model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = best_bert_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

bert_test_acc = accuracy_score(all_labels, all_preds)
bert_test_f1  = f1_score(all_labels, all_preds, average="weighted")

print("\n=== DistilBERT Test Performance ===")
print("Accuracy:", bert_test_acc)
print("Weighted F1:", bert_test_f1)
print("\nClassification Report (DistilBERT):\n")
print(classification_report(
    [label_encoder.classes_[i] for i in all_labels],
    [label_encoder.classes_[i] for i in all_preds]
))

# ================================
# HYBRID + TOP-K RANKING
# ================================

ml_model = joblib.load("best_aws_classifier.pkl")
tfidf    = joblib.load("tfidf_vectorizer.pkl")
le       = joblib.load("label_encoder.pkl")

bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert_aws_model").to(device)
bert_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert_aws_model")
bert_model.eval()

def ml_topk(text: str, k: int = 3) -> List[Tuple[str, float]]:
    """
    Return top-k (service, score) from the ML model.
    """
    clean = preprocess(text)
    if not clean:
        return []

    vec = tfidf.transform([clean])

    if hasattr(ml_model, "predict_proba"):
        proba = ml_model.predict_proba(vec)[0]
        classes = ml_model.classes_
        idx_sorted = np.argsort(proba)[::-1][:k]
        return [(classes[i], float(proba[i])) for i in idx_sorted]

    elif hasattr(ml_model, "decision_function"):
        scores = ml_model.decision_function(vec)
        if scores.ndim == 1:
            scores = scores.reshape(1, -1)
        scores = scores[0]
        exp_scores = np.exp(scores - scores.max())
        proba = exp_scores / exp_scores.sum()
        classes = ml_model.classes_
        idx_sorted = np.argsort(proba)[::-1][:k]
        return [(classes[i], float(proba[i])) for i in idx_sorted]

    else:
        label = ml_model.predict(vec)[0]
        return [(label, 0.5)]

def distilbert_topk(text: str, k: int = 3) -> List[Tuple[str, float]]:
    """
    Return top-k (service, score) from DistilBERT.
    """
    enc = bert_tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt",
    )
    ids  = enc["input_ids"].to(device)
    mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        out = bert_model(input_ids=ids, attention_mask=mask)
        logits = out.logits[0].cpu().numpy()

    exp_logits = np.exp(logits - logits.max())
    proba = exp_logits / exp_logits.sum()

    idx_sorted = np.argsort(proba)[::-1][:k]
    return [(le.classes_[i], float(proba[i])) for i in idx_sorted]

def hybrid_recommend_with_ranking(
    text: str,
    ml_threshold: float = 0.75,
    top_k: int = 3
) -> Tuple[str, str, float, List[Tuple[str, str, float]]]:
    """
    Returns:
        best_service, source_model, best_score,
        ranked_candidates = [(service, category, score), ...]
    """
    ml_candidates = ml_topk(text, k=top_k)

    if len(ml_candidates) == 0:
        # Fallback to DistilBERT only
        bert_candidates = distilbert_topk(text, k=top_k)
        ranked = [
            (svc, service_category_map.get(svc, "Unknown"), score)
            for svc, score in bert_candidates
        ]
        best_service, _, best_score = ranked[0]
        return best_service, "DistilBERT", best_score, ranked

    best_ml_service, best_ml_score = ml_candidates[0]

    if best_ml_score >= ml_threshold:
        source = "ML"
        raw_candidates = ml_candidates
    else:
        source = "DistilBERT"
        raw_candidates = distilbert_topk(text, k=top_k)

    ranked = [
        (svc, service_category_map.get(svc, "Unknown"), score)
        for svc, score in raw_candidates
    ]

    best_service, _, best_score = ranked[0]
    return best_service, source, best_score, ranked

def hybrid_recommend_aws_service(text: str, ml_threshold: float = 0.75) -> Tuple[str, str, float]:
    """
    Backward-compatible function (for Gradio, etc.)
    Returns single best service + source + best_score.
    """
    best_service, source, best_score, _ = hybrid_recommend_with_ranking(
        text, ml_threshold=ml_threshold, top_k=3
    )
    return best_service, source, best_score

# ================================
# INTERACTIVE CLI WITH SUBCATEGORY CHOICE
# ================================

print("\n================ AWS ARCHITECTURAL RECOMMENDER — INTERACTIVE MODE ================")
print("Type your AWS requirement below. Type 'exit' to stop.\n")

while True:
    user_text = input("Your Query: ").strip()

    if user_text.lower() == "exit":
        print("\nExiting AWS Recommender System. Goodbye!")
        break

    if len(user_text) < 3:
        print("⚠️ Please enter a valid requirement (at least 3 characters).\n")
        continue

    best_service, source, best_score, ranked_list = hybrid_recommend_with_ranking(
        user_text,
        ml_threshold=0.75,
        top_k=3
    )

    print("\n-----------------------------------------------")
    print(f"🔍 User Query: {user_text}")
    print(f"🤖 Primary Model Used: {source}")
    print("\nTop candidate services (ranked):")

    for idx, (svc, cat, score) in enumerate(ranked_list, start=1):
        print(f"  {idx}. {svc}  | Category: {cat}  | Score: {score:.3f}")

    print("\nBy default, the system will pick option 1.")
    print("If you want to override and choose a sub-service yourself,")
    print("enter 1 / 2 / 3 (depending on how many are shown), or press Enter to accept #1.")

    choice = input("Your choice [1-3 or Enter]: ").strip()

    selected_service = best_service
    selected_category = service_category_map.get(best_service, "Unknown")

    if choice.isdigit():
        idx = int(choice)
        if 1 <= idx <= len(ranked_list):
            selected_service, selected_category, selected_score = ranked_list[idx - 1]
            print(f"\n✅ You selected option {idx}.")
            best_score = selected_score  # update to user-selected score
        else:
            print("Invalid choice, falling back to top-ranked service.\n")
    else:
        print("No manual choice entered. Using top-ranked service.\n")

    print("=============== FINAL RECOMMENDATION ===============")
    print(f"🏷️  AWS Service: {selected_service}")
    print(f"📂 Category: {selected_category}")
    print(f"📊 Confidence Score (model space): {best_score:.3f}")
    print(f"🤖 Model Source: {source}")
    print("====================================================\n")


Using device: cuda
Original shape: (11000, 7)
Columns: ['requirement_text', 'primary_service', 'service_category', 'complexity_level', 'deployment_scenario', 'keywords_extracted', 'reference_url']

Service → Category map (sample):
  Amazon VPC  -->  Networking
  Amazon EFS  -->  Storage
  Amazon CloudFront  -->  Networking
  AWS Backup  -->  Storage
  Amazon ElastiCache  -->  Database
  Amazon Aurora  -->  Database
  AWS Fargate  -->  Compute
  AWS WAF  -->  Security
  Elastic Load Balancing  -->  Networking
  Amazon DynamoDB  -->  Database

After cleaning: (11000, 2)
After preprocessing: (11000, 3)
Number of classes: 20
Classes (sample): ['AWS Backup', 'AWS Fargate', 'AWS IAM', 'AWS KMS', 'AWS Lambda', 'AWS Secrets Manager', 'AWS WAF', 'Amazon Aurora', 'Amazon CloudFront', 'Amazon DynamoDB'] ...
Train size: 7700
Val size: 1650
Test size: 1650
TF-IDF shapes: (7700, 2134) (1650, 2134) (1650, 2134)

Training ML model: LinearSVC
LinearSVC Val Accuracy: 1.0000
LinearSVC Val F1-score: 1.000

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Epoch 1/5 =====
Epoch 1 Train Loss: 1.1230 | Train Acc: 0.7889
Val Loss: 0.0372 | Val Acc: 1.0000 | Val F1: 1.0000
New best DistilBERT model, saving checkpoint...

===== Epoch 2/5 =====
Epoch 2 Train Loss: 0.0269 | Train Acc: 1.0000
Val Loss: 0.0093 | Val Acc: 1.0000 | Val F1: 1.0000

===== Epoch 3/5 =====
Epoch 3 Train Loss: 0.0102 | Train Acc: 1.0000
Val Loss: 0.0048 | Val Acc: 1.0000 | Val F1: 1.0000

===== Epoch 4/5 =====
Epoch 4 Train Loss: 0.0062 | Train Acc: 1.0000
Val Loss: 0.0033 | Val Acc: 1.0000 | Val F1: 1.0000

===== Epoch 5/5 =====
Epoch 5 Train Loss: 0.0048 | Train Acc: 1.0000
Val Loss: 0.0029 | Val Acc: 1.0000 | Val F1: 1.0000

=== DistilBERT Test Performance ===
Accuracy: 1.0
Weighted F1: 1.0

Classification Report (DistilBERT):

                        precision    recall  f1-score   support

            AWS Backup       1.00      1.00      1.00       214
           AWS Fargate       1.00      1.00      1.00       173
               AWS IAM       1.00      1.00

KeyboardInterrupt: Interrupted by user